## This is new section, where we try to define queries in Natural Language.

Aperturedb has a property graph interface in llama_index repo. will try to use that to query the KG.

In [2]:
%pip install llama-index-llms-openai
%pip install --upgrade git+https://github.com/aperture-data/llama_index@kg_fixes#subdirectory=llama-index-integrations/graph_stores/llama-index-graph-stores-ApertureDB

You should consider upgrading via the '/Users/gsaluja/.virtualenvs/kgc/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/aperture-data/llama_index (to revision kg_fixes) to /private/var/folders/88/nch91zlx791cmww068r1vkf40000gn/T/pip-req-build-dlsiopx7
  Running command git clone --filter=blob:none --quiet https://github.com/aperture-data/llama_index /private/var/folders/88/nch91zlx791cmww068r1vkf40000gn/T/pip-req-build-dlsiopx7
  Running command git checkout -b kg_fixes --track origin/kg_fixes
  Switched to a new branch 'kg_fixes'
  branch 'kg_fixes' set up to track 'origin/kg_fixes'.
  Resolved https://github.com/aperture-data/llama_index to commit f8c944b91835226a1fb48f729c0f72e4e85afb63
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 KB 3.2 MB/s e

In [1]:
from typing import Literal
from llama_index.core import PropertyGraphIndex

from llama_index.core import StorageContext
from llama_index.graph_stores.ApertureDB import ApertureDBGraphStore

from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

In [2]:
entities = Literal["MOVIE", "GENRE", "KEYWORD", "PROFESSIONAL"]
relations = Literal["CAST", "CREW", "HAS_GENRE", "HAS_KEYWORD"]

validation_schema = {
    "PERSON": ["HAS", "PART_OF", "WORKED_ON", "WORKED_WITH", "WORKED_AT"],
    "PLACE": ["HAS", "PART_OF", "WORKED_AT"],
    "ORGANIZATION": ["HAS", "PART_OF", "WORKED_WITH"],
    "CONCEPT": ["HAS", "PART_OF", "WORKED_ON", "WORKED_WITH"],
}


In [3]:
kg_extractor = SchemaLLMPathExtractor(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0),
    possible_entities=entities,
    possible_relations=relations,
    possible_entity_props=["name", "budget"],
)

graph_store = ApertureDBGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

index = PropertyGraphIndex.from_existing(
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    property_graph_store=graph_store,
    show_progress=True,
)

query_engine = index.as_query_engine()


In [4]:
def run_queries(query_engine):
    query_str = [
        ("Which movies is Tom Hanks associated with? Give me a list of movies.", True),
        ("Give me a list of movies starring Tom Hanks and Meg Ryan.", True),
        ("What is the genre of the movie 'Forrest Gump'?", True),
        ("Which other actors has Tom Hanks worked with?", True),
        ("What is the budget of the movie 'Forrest Gump'?", False),
        (
            "List all the crew members of the movie 'Forrest Gump', which were not actors.",
            False,
        ),
    ]
    for qs, is_correct in query_str:
        response = query_engine.query(qs)
        print(f"{qs=}")
        print(f"{response.response=}")
        print("=" * 50)


run_queries(query_engine)

qs='Which movies is Tom Hanks associated with? Give me a list of movies.'
response.response="Extremely Loud & Incredibly Close, Philadelphia, Who Killed the Electric Car?, Saving Private Ryan, Apollo 13, You've Got Mail, The Green Mile, Toy Story 3, Evan Almighty, The Polar Express, City of Ember, Saving Mr. Banks, The Ladykillers, Splash, Radio Flyer, Angels & Demons, Forrest Gump, Captain Phillips, Catch Me If You Can, Mamma Mia!, That Thing You Do!, Toy Story, The Da Vinci Code, Where the Wild Things Are, Larry Crowne, Cloud Atlas, Toy Story 2, Cast Away, Big, My Big Fat Greek Wedding 2."
qs='Give me a list of movies starring Tom Hanks and Meg Ryan.'
response.response="You've Got Mail"
qs="What is the genre of the movie 'Forrest Gump'?"
response.response='Drama'
qs='Which other actors has Tom Hanks worked with?'
response.response='Tom Hanks has worked with actors such as Meg Ryan, Tim Allen, Robin Wright, Matt Damon, Leonardo DiCaprio, and Tom Hanks has worked with actors such as Me